In [ ]:
#create a basic Settings class to import the api key from env file and create a sales agent from it
from pydantic_ai import Agent
from pydantic_settings import BaseSettings
from pydantic import Field
import os
from dotenv import load_dotenv

load_dotenv()

class Settings(BaseSettings):
    groq_api_key: str = Field(alias="GROQ_API_KEY")
    serpapi_key: str = Field(alias="SERPAPI_KEY")
    openai_api_key: str = Field(alias="OPENAI_API_KEY")
    openrouter_api_key: str = Field(alias="OPENROUTER_API_KEY")
    openrouter_api_base: str = Field(alias="OPENROUTER_API_BASE")
    model_config = {"env_file": ".env", "extra": "ignore"}

settings = Settings()
os.environ["GROQ_API_KEY"] = settings.groq_api_key
os.environ["OPENAI_API_KEY"] = settings.openai_api_key
os.environ["SERPAPI_KEY"]= settings.serpapi_key
os.environ["OPENROUTER_API_KEY"]= settings.openrouter_api_key
os.environ["OPENROUTER_API_BASE"]= settings.openrouter_api_base

sales_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    system_prompt="You are a data analyst."
)

result = await sales_agent.run("What metrics should I track?")
print(result.output)


As a data analyst, the key metrics you should track depend on your specific goals, industry, and objectives. However, here are some general metrics across various categories that are commonly tracked:

**Business Metrics**

1. **Revenue**: Total sales, revenue growth rate, revenue per user (RPU)
2. **Customer Acquisition Cost (CAC)**: Cost of acquiring a new customer
3. **Customer Lifetime Value (CLV)**: Average value of a customer over their lifetime
4. **Conversion Rate**: Percentage of users who complete a desired action (e.g., make a purchase, submit a form)
5. **Return on Investment (ROI)**: Return on investment for specific campaigns or initiatives

**User Engagement Metrics**

1. **Website Traffic**: Number of visitors, page views, bounce rate
2. **Engagement Metrics**: Average session duration, pages per session, time spent on site
3. **Social Media Metrics**: Followers, engagement rate, reach, clicks-per-click-through-rate
4. **Email Open and Click-Through Rates**: Email open 

In [38]:
#creating agents with differnet models and tweaking the model settings
groq_intern_agent=Agent(
    "groq:llama-3.1-8b-instant", 
    system_prompt="You are a data science intern at an AI Health startup. Provide clear, actionable insights based on the data provided", model_settings={'temperature': 0.3, 'max_tokens':500}
)

question=await groq_intern_agent.run("What are some of the data formats for health data in India? Answer in 2-3 lines")

print(question.output)


In India, common data formats for health data include:

1. **CDAC's National Health Stack (NHS)**: A standardized format for electronic health records (EHRs) and health data exchange.
2. **HL7 (Health Level Seven)**: A widely used format for exchanging health data between different systems and applications.
3. **FHIR (Fast Healthcare Interoperability Resources)**: An emerging standard for exchanging health data in a machine-readable format.


In [60]:
#using built in search tool with our sales agent
import os
from pydantic_ai import Agent
from serpapi import GoogleSearch


def web_search_tool(query: str) -> dict:
    params={
        "engine": "google",
        "q" : query,
        "api_key": os.environ["SERPAPI_KEY"]
    }
    
    result=GoogleSearch(params).get_dict()
    return result.get("organic_results")

market_research_agent=Agent(
    'groq:llama-3.1-8b-instant',
    tools=[web_search_tool],
    system_prompt='You are a market analyst. Use the web search tool to find the current information.'
) 

information_result= await market_research_agent.run("What is the current stock price of CMS Info Systems Ltd?")

print(os.getenv("SERPAPI_KEY"))
print(information_result.output)

a0b338f764eb2807e764e630b280f5760feb5ae2271212d868750ee5e2e41331
The current stock price of CMS Info Systems Ltd is ₹362.50 INR.


In [72]:
#using code execution for mathematical analysis and data processing
from pydantic_ai import Agent, CodeExecutionTool
data_analysiscalculation_agent = Agent(
    'groq:llama-3.1-8b-instant',
    system_prompt='"You are a data analyst. Use code to perform calculations and create visualizations. " "Show your work and provide clear explanations." '
)
calculation_output=await data_analysiscalculation_agent.run("I have sales data for Q1 2025:- North region: 150 laptops at $1,500 each, 320 phones at $500 each- South region: 89 laptops at $1,500 each, 245 phones at $500 each - East region: 201 laptops at $1,500 each, 180 phones at $500 each- West region: 167 laptops at $1,500 each, 290 phones at $500 eachCalculate total revenue by region and create a simple chart showing the results.")
print(calculation_output.output)

### Calculating Total Revenue by Region

To solve this problem, we'll write a Python script using pandas for data manipulation and matplotlib for creating the chart.

```python
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Define data
data = {
    'Region': ['North', 'South', 'East', 'West'],
    'Laptops': [150, 89, 201, 167],
    'Phones': [320, 245, 180, 290],
    'Price_Laptop': 1500,
    'Price_Phone': 500
}

# Create a DataFrame
df = pd.DataFrame(data)

# Calculate total revenue by region
df['Revenue_Laptop'] = df['Laptops'] * df['Price_Laptop']
df['Revenue_Phone'] = df['Phones'] * df['Price_Phone']
df['Total_Revenue'] = df['Revenue_Laptop'] + df['Revenue_Phone']

# Print the DataFrame
print("Total Revenue by Region:")
print(df[['Region', 'Revenue_Laptop', 'Revenue_Phone', 'Total_Revenue']])

# Create a bar chart to visualize the results
plt.figure(figsize=(8, 6))
plt.bar(df['Region'], df['Total_Revenue'])
plt.xlabel('Region')
plt.ylabel('Tot

In [76]:
#using structured outputs (output_type feature available only in anthropic and openai models)
import json
from pydantic import BaseModel, ValidationError
from pydantic_ai import Agent

class SalesInsight(BaseModel):
    total_revenue: float
    best_performing_region: str
    worst_performing_region: str
    recommendation: str

# Groq model agent (no output_type)
analysis_agent = Agent(
    "groq:llama-3.1-8b-instant",
    system_prompt="""
You are a sales analyst.
Return your answer *only* in the following JSON format:

{
  "total_revenue": number,
  "best_performing_region": "string",
  "worst_performing_region": "string",
  "recommendation": "string"
}

Do not add explanations, markdown, sentences, or commentary.
Only valid JSON.
"""
)

# User prompt
text_data = """
Q1 2025 sales data:
- North: $385,000 revenue (470 units)
- South: $256,000 revenue (334 units)
- East: $391,500 revenue (381 units)
- West:  $395,500 revenue (457 units)
Analyze this data and provide insights.
"""

# Run Groq model
response = await analysis_agent.run(text_data)
raw_json = response.output.strip()

# Parse into Pydantic model
try:
    data = SalesInsight(**json.loads(raw_json))
except (json.JSONDecodeError, ValidationError) as e:
    raise ValueError(f"Model returned invalid JSON:\n{raw_json}") from e

# Output
print(f"Total Revenue: ${data.total_revenue:,.0f}")
print(f"Best Region: {data.best_performing_region}")
print(f"Worst Region: {data.worst_performing_region}")
print(f"Recommendation: {data.recommendation}")


Total Revenue: $1,428
Best Region: West
Worst Region: South
Recommendation: Increase marketing efforts in North and South regions to boost sales


In [ ]:
#accessing the message history
from pydantic_ai import Agent
sales_agent = Agent(
    'groq:llama-3.1-8b-instant',
    system_prompt="You are a sales analyst. Provide clear, concise analysis."
)

# First question
result1 = await sales_agent.run("What are the main KPIs I should track for Q1 2025 sales?")

print(result1.output)

# Conversation history
all_messages = result1.all_messages()
print(f"Total messages in conversation: {len(all_messages)}")

new_messages = result1.new_messages()
print(f"New messages from this run: {len(new_messages)}")


To effectively measure Q1 2025 sales performance, here are the key performance indicators (KPIs) to track:

1. **Sales Revenue**: Total sales generated during Q1 2025, compared to the same period last year. This KPI helps you understand the overall growth or decline in sales.

2. **Conversion Rate**: The percentage of leads that become paying customers. This KPI measures the effectiveness of your sales funnel and can help you identify areas for improvement.

3. **Customer Acquisition Cost (CAC)**: The cost of acquiring a new customer, including marketing and sales expenses. This KPI helps you understand the profitability of your customer base.

4. **Customer Retention Rate**: The percentage of customers retained from one quarter to the next. This KPI measures the loyalty and satisfaction of your existing customer base.

5. **Average Order Value (AOV)**: The average amount spent by a customer in a single transaction. This KPI helps you understand the average revenue generated per custom

In [ ]:
#Continuing conversations with message history (To build on previous conversations, pass the message history to your next agent run)
from pydantic_ai import Agent

sales_agent = Agent(
    'groq:llama-3.1-8b-instant',
    system_prompt="You are a sales analyst. Provide clear, concise analysis."
)

# First question
result1 = await sales_agent.run("What are the main KPIs I should track for Q1 2025 sales?")
print(result1.output)  

# Continue the conversation
result2 = await sales_agent.run(
    "How should I calculate conversion rates for each of those KPIs?",
    message_history=result1.all_messages()  # ✅ pass conversation history
)
print(result2.output)

print(f"Full conversation length: {len(result2.all_messages())}")

# Follow-up question using total conversation context
result3 = await sales_agent.run(
    "Which of these metrics would be most important for a monthly executive report?",
    message_history=result2.all_messages()
)
print(result3.output)


To effectively track sales performance for Q1 2025, consider the following key performance indicators (KPIs):

1. **Sales Revenue Growth Rate**: Measure the year-over-year increase in sales revenue to evaluate the overall growth of the business.
2. **Average Order Value (AOV)**: Track the average value of each order to understand the mix of high-value and low-value sales and identify opportunities for increasing revenue.
3. **Conversion Rate**: Monitor the percentage of website visitors or leads converted into sales to optimize the sales funnel and improve revenue generation.
4. **Sales Cycle Length**: Measure the average time it takes to close a sale to identify bottlenecks and optimize the sales process.
5. **Customer Acquisition Cost (CAC)**: Track the cost of acquiring new customers, including marketing and sales expenses, to ensure the revenue generated justifies the investment.
6. **Customer Lifetime Value (CLV)**: Evaluate the total value of a customer throughout their lifetime 

In [ ]:
#Storing and loading messages to JSON
import json
from pydantic_ai import Agent

# Create Groq Agent
sales_agent = Agent(
    'groq:llama-3.1-8b-instant',
    system_prompt="You are a sales analyst. Provide clear, concise analysis."
)

# Conversation Steps
result1 =await sales_agent.run("What are the main KPIs I should track for Q1 2025 sales?")
result2 = await sales_agent.run(
    "How should I calculate conversion rates for each of those KPIs?",
    message_history=result1.all_messages()
)
result3 =await sales_agent.run(
    "Which of these metrics would be most important for a monthly executive report?",
    message_history=result2.all_messages()
)

# ✅ Use .text instead of .output for display
print(result3.output)

# ✅ Export full conversation history (works identical for Groq)
conversation_json = result3.all_messages_json()

# Save to file
with open('sales_analysis_conversation.json', 'wb') as f:
    f.write(conversation_json)

print("✅ Conversation saved to sales_analysis_conversation.json")

# Load conversation back later
with open('sales_analysis_conversation.json', 'rb') as f:
    loaded_conversation = f.read()

print(f"Loaded conversation size: {len(loaded_conversation)} bytes")


For a monthly executive report, I recommend focusing on a few key metrics that provide a comprehensive view of sales performance. Based on the KPIs we've discussed, I'd suggest highlighting the following metrics:

1. **Revenue Growth Rate**: This metric provides a clear understanding of sales performance over time and helps executives track progress towards revenue goals.
2. **Sales Funnel Conversion Rate**: By highlighting the conversion rates at different stages of the sales funnel (e.g., lead to qualified lead, demo to proposal), executives can quickly identify areas where the sales process may be breaking down or where there are opportunities for improvement.
3. **Revenue per Sales Dollar Spent**: This metric provides insight into the efficiency of sales spend and helps executives optimize their investment in sales and marketing initiatives.

Consider including the following data points in your monthly executive report:

* Revenue growth rate: Year-over-year (YoY) or quarter-over-q

In [ ]:
from pydantic_ai import Agent

agent = Agent(
    model=""
)